In [4]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv('../train_wo_missing_values.csv')
test = pd.read_csv('../test.csv')

In [3]:
data.drop(index=[523,1298], inplace=True)

In [6]:
from transform import transform_dataset

In [7]:
x, y = transform_dataset(data)

In [8]:
x.to_csv('../train_transf.csv', index=False)
y.to_csv('../test_transf.csv', index=False, header=False)

In [580]:
rand_variable = np.random.rand(x.shape[0])*100

In [581]:
x = x.reset_index(drop=True)

In [582]:
x = pd.concat([x, pd.Series(rand_variable, name='Rand_Var')], axis=1)

In [572]:
pca_var = pd.read_csv('../x12d.csv', prefix='pca')

In [573]:
pca_var.shape

(1458, 12)

In [574]:
x = pd.concat([x, pca_var], axis=1)

In [544]:
x.head()

,LotFrontage,LotArea,YearBuilt,MasVnrArea,TotalBsmtSF,GrLivArea,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,...,2,3,4,5,6,7,8,9,10,11
0,13.754179,9.041922,2003,196.0,856,1710,2003.0,211.355516,0,61,...,0.489574,-0.070740,0.080191,-0.396822,0.819830,1.454054,-0.530810,-0.314133,0.675321,-0.309865
1,15.446997,9.169518,1976,0.0,1262,1262,1976.0,183.057985,298,0,...,-0.752644,0.198471,-0.047932,0.915364,-0.400408,-0.380815,-0.143054,0.270766,-1.185431,0.028497
2,14.107516,9.328123,2001,162.0,920,1786,2001.0,230.170899,0,42,...,0.333702,0.316951,0.334385,1.192629,0.223092,0.657625,-0.554016,0.348407,-0.566953,-0.760743
3,13.146282,9.164296,1915,0.0,756,1717,1998.0,240.681574,0,35,...,-0.382669,-1.279727,3.367146,-1.145491,-0.414045,-0.896113,-0.067543,0.869701,0.666970,-1.126284
4,15.870407,9.565214,2000,350.0,1145,2198,2000.0,298.887194,192,84,...,0.377549,0.281931,0.375355,1.157149,0.335261,0.444143,-0.367786,-0.198315,-0.038292,-0.898506


In [36]:
from sklearn import ensemble
randomForest = ensemble.RandomForestRegressor()

In [37]:
randomForest.set_params(n_estimators=100, oob_score=True)
randomForest.fit(x, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [38]:
#param_test = {'max_features':range(7,20,2),
#              'min_samples_split':range(80,301,20), 'min_samples_leaf':range(5,66,10),
#              'max_depth':range(5,16,2), 'min_samples_split':range(20,101,20),
#              'n_estimators':range(20,1001,10)}
#gsearch = GridSearchCV(estimator = randomForest,
#param_grid = param_test, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=10)
#gsearch.fit(x, y)
#gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_

In [39]:
randomForest.score(x, y)

0.981003380347395

In [40]:
randomForest.oob_score_

0.8642604799196224

In [41]:
feature_importance = list(zip(x.columns, randomForest.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance = np.array(feature_importance, dtype=dtype)
feature_sort = np.sort(feature_importance, order='importance')[::-1]
feature_sort[:15]

array([(b'LotGrBsmtArea', 0.36840096), (b'YearBuilt', 0.14253149),
       (b'GrLivArea', 0.13529386), (b'ExterQual_TA', 0.0705841 ),
       (b'GarageCars', 0.03252303), (b'CentralAir_Y', 0.02736767),
       (b'TotalBsmtSF', 0.01742461), (b'GarageArea', 0.01641494),
       (b'1stFlrSF', 0.01428137), (b'YearRemodAdd', 0.01344729),
       (b'BsmtFinSF1', 0.01081224), (b'2ndFlrSF', 0.00847521),
       (b'KitchenQual_TA', 0.00712818), (b'BsmtUnfSF', 0.00667557),
       (b'LotFrontage', 0.00540119)],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [42]:
[i for (i, j) in feature_sort[0:15]]

[b'LotGrBsmtArea',
 b'YearBuilt',
 b'GrLivArea',
 b'ExterQual_TA',
 b'GarageCars',
 b'CentralAir_Y',
 b'TotalBsmtSF',
 b'GarageArea',
 b'1stFlrSF',
 b'YearRemodAdd',
 b'BsmtFinSF1',
 b'2ndFlrSF',
 b'KitchenQual_TA',
 b'BsmtUnfSF',
 b'LotFrontage']

In [136]:
data = pd.read_csv('../train_wo_missing_values.csv')
x_, y_ = data.loc[:, data.columns != 'SalePrice'], data.loc[:,'SalePrice']

In [139]:
from scipy import stats

In [137]:
categorical_columns = ['MSSubClass','MSZoning','Street','LotShape','LandContour','LotConfig',
                         'LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle',
                         'OverallQual','OverallCond','RoofStyle','RoofMatl','Exterior1st','Exterior2nd',
                         'MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtExposure',
                         'BsmtFinType1','Heating','HeatingQC','CentralAir','Electrical','KitchenQual',
                         'Functional','FireplaceQu','GarageType','GarageFinish','PavedDrive',
                         'SaleType','SaleCondition', 'Utilities']

In [140]:
continuous_columns = x_.drop(columns=categorical_columns).columns

for column in continuous_columns:
    k, p = stats.normaltest(x_[column])
    if p < 0.05:
        x_[column] = np.log(x_[column] + 1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
dummies = pd.get_dummies(x_.loc[:,categorical_columns], drop_first=True)

In [24]:
dummies.head()

,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Utilities_NoSeWa
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0


In [25]:
x = pd.concat([x_.drop(columns=categorical_columns), dummies], axis=1)

In [35]:
x['LotGrBsmtArea'] = x['GrLivArea'] * x['LotArea'] * x['TotalBsmtSF']

In [26]:
y = np.log(y_)

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Utilities_NoSeWa
0,65.0,8450,2003,2003,196.0,706,0,150,856,856,...,0,0,0,1,0,0,0,1,0,0
1,80.0,9600,1976,1976,0.0,978,0,284,1262,1262,...,0,0,0,1,0,0,0,1,0,0
2,68.0,11250,2001,2002,162.0,486,0,434,920,920,...,0,0,0,1,0,0,0,1,0,0
3,60.0,9550,1915,1970,0.0,216,0,540,756,961,...,0,0,0,1,0,0,0,0,0,0
4,84.0,14260,2000,2000,350.0,655,0,490,1145,1145,...,0,0,0,1,0,0,0,1,0,0


In [43]:
from xgboost import XGBRegressor

In [44]:
from sklearn.preprocessing import StandardScaler

In [49]:
from sklearn.model_selection import GridSearchCV

In [45]:
xgboost = XGBRegressor(learning_rate=0.01, n_estimators=1000)
xgboost.fit(x, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [46]:
xgboost.score(x, y)

0.9457035583231194

In [47]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

In [50]:
gsearch1 = GridSearchCV(xgboost, param_grid=param_test1, scoring='neg_mean_squared_error', n_jobs=-1, iid=False, cv=10)
gsearch1.fit(x, y)

GridSearchCV(cv=10, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid={'max_depth': range(3, 10, 2), 'min_child_weight': range(1, 6, 2)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [51]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.01839, std: 0.00503, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: -0.01857, std: 0.00480, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: -0.01860, std: 0.00472, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: -0.01733, std: 0.00478, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: -0.01740, std: 0.00432, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: -0.01754, std: 0.00417, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: -0.01739, std: 0.00480, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: -0.01743, std: 0.00432, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: -0.01737, std: 0.00406, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: -0.01752, std: 0.00503, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: -0.01762, std: 0.00415, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: -0.01755, std: 0.00450, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 5, 'min_chil

In [52]:
gsearch1.best_estimator_.score(x, y)

0.9750444141812591

In [62]:
np.sqrt(np.sum((np.array(y)-gsearch1.best_estimator_.predict(x))**2))

2.4103271197185485

In [710]:
param_test2 = {
 'max_depth':[6,7,8],
 'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBRegressor(learning_rate=0.01, n_estimators=1000, max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test2, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=10)
gsearch2.fit(x,y)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.01754, std: 0.00396, params: {'max_depth': 6, 'min_child_weight': 4},
  mean: -0.01756, std: 0.00391, params: {'max_depth': 6, 'min_child_weight': 5},
  mean: -0.01762, std: 0.00391, params: {'max_depth': 6, 'min_child_weight': 6},
  mean: -0.01757, std: 0.00390, params: {'max_depth': 7, 'min_child_weight': 4},
  mean: -0.01754, std: 0.00396, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: -0.01758, std: 0.00396, params: {'max_depth': 7, 'min_child_weight': 6},
  mean: -0.01753, std: 0.00391, params: {'max_depth': 8, 'min_child_weight': 4},
  mean: -0.01755, std: 0.00399, params: {'max_depth': 8, 'min_child_weight': 5},
  mean: -0.01753, std: 0.00397, params: {'max_depth': 8, 'min_child_weight': 6}],
 {'max_depth': 8, 'min_child_weight': 6},
 -0.01752515006443246)

In [711]:
gsearch2.best_estimator_.score(x, y)

0.9842725719338103

In [714]:
param_test2b = {
 'max_depth':[9,10,11],
 'min_child_weight':[6,8,10,12]
}
gsearch2b = GridSearchCV(estimator = XGBRegressor(learning_rate=0.01, n_estimators=1000,max_depth=8,max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test2b, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=10)
gsearch2b.fit(x, y)
gsearch2b.grid_scores_, gsearch2b.best_params_, gsearch2b.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.01762, std: 0.00395, params: {'max_depth': 9, 'min_child_weight': 6},
  mean: -0.01744, std: 0.00378, params: {'max_depth': 9, 'min_child_weight': 8},
  mean: -0.01756, std: 0.00389, params: {'max_depth': 9, 'min_child_weight': 10},
  mean: -0.01772, std: 0.00384, params: {'max_depth': 9, 'min_child_weight': 12},
  mean: -0.01760, std: 0.00385, params: {'max_depth': 10, 'min_child_weight': 6},
  mean: -0.01751, std: 0.00377, params: {'max_depth': 10, 'min_child_weight': 8},
  mean: -0.01756, std: 0.00388, params: {'max_depth': 10, 'min_child_weight': 10},
  mean: -0.01769, std: 0.00379, params: {'max_depth': 10, 'min_child_weight': 12},
  mean: -0.01765, std: 0.00388, params: {'max_depth': 11, 'min_child_weight': 6},
  mean: -0.01755, std: 0.00379, params: {'max_depth': 11, 'min_child_weight': 8},
  mean: -0.01760, std: 0.00383, params: {'max_depth': 11, 'min_child_weight': 10},
  mean: -0.01771, std: 0.00388, params: {'max_depth': 11, 'min_child_weight': 12}],
 {'max_depth'

In [716]:
gsearch2b.best_estimator_.score(x, y)

0.9841403694051859

In [63]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBRegressor(learning_rate=0.01, n_estimators=1000,max_depth=5, min_child_weight=1, subsample=0.8, random_state=10),
param_grid = param_test3, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=10)
gsearch3.fit(x, y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.01708, std: 0.00481, params: {'gamma': 0.0},
  mean: -0.01778, std: 0.00480, params: {'gamma': 0.1},
  mean: -0.01906, std: 0.00488, params: {'gamma': 0.2},
  mean: -0.02029, std: 0.00489, params: {'gamma': 0.3},
  mean: -0.02138, std: 0.00499, params: {'gamma': 0.4}],
 {'gamma': 0.0},
 -0.017081281265834712)

In [66]:
feature_importance = list(zip(x.columns, gsearch3.best_estimator_.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance = np.array(feature_importance, dtype=dtype)
feature_sort = np.sort(feature_importance, order='importance')[::-1]
feature_sort[:15]

array([(b'LotFrontage', 0.07589439), (b'LotArea', 0.06080695),
       (b'YearBuilt', 0.05057721), (b'LotGrBsmtArea', 0.04343354),
       (b'GrLivArea', 0.04251914), (b'BsmtUnfSF', 0.03909018),
       (b'BsmtFinSF1', 0.03526117), (b'GarageArea', 0.03468968),
       (b'YearRemodAdd', 0.03011773), (b'1stFlrSF', 0.02863184),
       (b'TotalBsmtSF', 0.02726026), (b'OpenPorchSF', 0.02080238),
       (b'GarageYrBlt', 0.02063093), (b'MoSold', 0.02057378),
       (b'WoodDeckSF', 0.01903075)],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [67]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBRegressor(learning_rate=0.01, n_estimators=1000, max_depth=5, min_child_weight=1, subsample=0.8, random_state=10,max_features=7),
param_grid = param_test4, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=10)
gsearch4.fit(x, y)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.01669, std: 0.00488, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: -0.01677, std: 0.00505, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: -0.01675, std: 0.00456, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: -0.01694, std: 0.00473, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: -0.01650, std: 0.00474, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: -0.01672, std: 0.00489, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: -0.01694, std: 0.00471, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: -0.01688, std: 0.00461, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: -0.01660, std: 0.00469, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: -0.01689, std: 0.00479, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: -0.01692, std: 0.00476, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: -0.01707, std: 0.00487, params: {'colsample_bytree': 0.8,

In [68]:
gsearch4.best_estimator_.score(x, y)

0.9745673075008306

In [69]:
param_test5 = {
 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}
gsearch5 = GridSearchCV(estimator = XGBRegressor(learning_rate=0.01, n_estimators=1000,max_depth=5,min_child_weight=1, colsample_bytree=0.7, subsample=0.6, random_state=10),
param_grid = param_test5, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=10)
gsearch5.fit(x, y)
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.01650, std: 0.00474, params: {'reg_alpha': 0},
  mean: -0.01648, std: 0.00474, params: {'reg_alpha': 0.001},
  mean: -0.01647, std: 0.00475, params: {'reg_alpha': 0.005},
  mean: -0.01650, std: 0.00480, params: {'reg_alpha': 0.01},
  mean: -0.01655, std: 0.00473, params: {'reg_alpha': 0.05}],
 {'reg_alpha': 0.005},
 -0.01647236110034047)

In [70]:
gsearch5.best_estimator_.score(x, y)

0.9743529795551161

In [71]:
feature_importance = list(zip(x.columns, gsearch5.best_estimator_.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance = np.array(feature_importance, dtype=dtype)
feature_sort = np.sort(feature_importance, order='importance')[::-1]
feature_sort[:15]

array([(b'LotFrontage', 0.06753598), (b'LotArea', 0.06132366),
       (b'YearBuilt', 0.04594661), (b'GrLivArea', 0.04551606),
       (b'LotGrBsmtArea', 0.04496248), (b'BsmtUnfSF', 0.03672038),
       (b'BsmtFinSF1', 0.03512117), (b'GarageArea', 0.03346045),
       (b'1stFlrSF', 0.03333743), (b'YearRemodAdd', 0.03063107),
       (b'TotalBsmtSF', 0.03013901), (b'GarageYrBlt', 0.02429573),
       (b'OpenPorchSF', 0.02208144), (b'2ndFlrSF', 0.01912904),
       (b'MoSold', 0.01845245)],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [72]:
xgboost = gsearch5.best_estimator_

In [115]:
np.sqrt(np.sum((np.array(y)-xgboost.predict(x))**2))

2.4434899755604422

In [73]:
feature_importance_xgboost = list(zip(x.columns, xgboost.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance_xgboost = np.array(feature_importance_xgboost, dtype=dtype)
feature_sort_xgboost = np.sort(feature_importance_xgboost, order='importance')[::-1]

In [74]:
feature_sort_xgboost[0:10]

array([(b'LotFrontage', 0.06753598), (b'LotArea', 0.06132366),
       (b'YearBuilt', 0.04594661), (b'GrLivArea', 0.04551606),
       (b'LotGrBsmtArea', 0.04496248), (b'BsmtUnfSF', 0.03672038),
       (b'BsmtFinSF1', 0.03512117), (b'GarageArea', 0.03346045),
       (b'1stFlrSF', 0.03333743), (b'YearRemodAdd', 0.03063107)],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [75]:
[i for (i, j) in feature_sort_xgboost[0:10]]

[b'LotFrontage',
 b'LotArea',
 b'YearBuilt',
 b'GrLivArea',
 b'LotGrBsmtArea',
 b'BsmtUnfSF',
 b'BsmtFinSF1',
 b'GarageArea',
 b'1stFlrSF',
 b'YearRemodAdd']

In [76]:
from sklearn.ensemble import GradientBoostingRegressor

In [77]:
gbm = GradientBoostingRegressor(n_estimators=10000, max_features=int(np.sqrt(x.shape[1])), subsample=0.8, learning_rate=0.001)
gbm.fit(x, y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.001, loss='ls', max_depth=3, max_features=15,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=10000, presort='auto', random_state=None,
             subsample=0.8, verbose=0, warm_start=False)

In [78]:
feature_importance_gbm = list(zip(x.columns, gbm.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance_gbm = np.array(feature_importance_gbm, dtype=dtype)
feature_sort_gbm = np.sort(feature_importance_gbm, order='importance')[::-1]

In [79]:
feature_sort_gbm[:10]

array([(b'LotGrBsmtArea', 0.06512314), (b'GrLivArea', 0.06483426),
       (b'1stFlrSF', 0.03836505), (b'BsmtFinSF1', 0.03352232),
       (b'TotalBsmtSF', 0.03239922), (b'GarageArea', 0.02671204),
       (b'2ndFlrSF', 0.02523467), (b'YearRemodAdd', 0.02521222),
       (b'YearBuilt', 0.02373082), (b'GarageYrBlt', 0.0227098 )],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [80]:
[i for (i, j) in feature_sort_gbm[0:10]]

[b'LotGrBsmtArea',
 b'GrLivArea',
 b'1stFlrSF',
 b'BsmtFinSF1',
 b'TotalBsmtSF',
 b'GarageArea',
 b'2ndFlrSF',
 b'YearRemodAdd',
 b'YearBuilt',
 b'GarageYrBlt']

In [81]:
gbm = GradientBoostingRegressor(n_estimators=10000, max_features=int(np.sqrt(x.shape[1])), subsample=0.8, learning_rate=0.1, min_samples_split=20, min_samples_leaf=10, max_depth=6)
gbm.fit(x, y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=6, max_features=15,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=10,
             min_samples_split=20, min_weight_fraction_leaf=0.0,
             n_estimators=10000, presort='auto', random_state=None,
             subsample=0.8, verbose=0, warm_start=False)

In [82]:
gbm.score(x, y)

0.9999994798442423

In [83]:
param_test1 = {'n_estimators':range(20,1001,10)}

In [85]:
gsearch1 = GridSearchCV(gbm, param_grid=param_test1, scoring='neg_mean_squared_error', n_jobs=-1, iid=False, cv=10)
gsearch1.fit(x, y)

GridSearchCV(cv=10, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=6, max_features=15,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=10,
             min_samples_split=20, min_weight_fraction_leaf=0.0,
             n_estimators=10000, presort='auto', random_state=None,
             subsample=0.8, verbose=0, warm_start=False),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid={'n_estimators': range(20, 1001, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [86]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.02854, std: 0.00578, params: {'n_estimators': 20},
  mean: -0.02235, std: 0.00564, params: {'n_estimators': 30},
  mean: -0.02014, std: 0.00480, params: {'n_estimators': 40},
  mean: -0.01906, std: 0.00545, params: {'n_estimators': 50},
  mean: -0.01875, std: 0.00513, params: {'n_estimators': 60},
  mean: -0.01792, std: 0.00455, params: {'n_estimators': 70},
  mean: -0.01801, std: 0.00491, params: {'n_estimators': 80},
  mean: -0.01777, std: 0.00527, params: {'n_estimators': 90},
  mean: -0.01702, std: 0.00373, params: {'n_estimators': 100},
  mean: -0.01754, std: 0.00524, params: {'n_estimators': 110},
  mean: -0.01696, std: 0.00478, params: {'n_estimators': 120},
  mean: -0.01682, std: 0.00504, params: {'n_estimators': 130},
  mean: -0.01675, std: 0.00467, params: {'n_estimators': 140},
  mean: -0.01676, std: 0.00409, params: {'n_estimators': 150},
  mean: -0.01728, std: 0.00490, params: {'n_estimators': 160},
  mean: -0.01727, std: 0.00477, params: {'n_estimators': 170},


In [87]:
gsearch1.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=6, max_features=15,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=10,
             min_samples_split=20, min_weight_fraction_leaf=0.0,
             n_estimators=430, presort='auto', random_state=None,
             subsample=0.8, verbose=0, warm_start=False)

In [88]:
feature_importance_gbm = list(zip(x.columns, gsearch1.best_estimator_.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance_gbm = np.array(feature_importance_gbm, dtype=dtype)
feature_sort_gbm = np.sort(feature_importance_gbm, order='importance')[::-1]
feature_sort_gbm[:15]

array([(b'GrLivArea', 0.04866399), (b'1stFlrSF', 0.04477908),
       (b'GarageArea', 0.04418076), (b'BsmtUnfSF', 0.04162607),
       (b'LotGrBsmtArea', 0.04028268), (b'LotArea', 0.04004367),
       (b'GarageYrBlt', 0.03294611), (b'TotalBsmtSF', 0.03151499),
       (b'LotFrontage', 0.03126566), (b'YearBuilt', 0.03110655),
       (b'OpenPorchSF', 0.03079572), (b'YearRemodAdd', 0.02916912),
       (b'BsmtFinSF1', 0.02895574), (b'MoSold', 0.02094234),
       (b'MasVnrArea', 0.01858362)],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [89]:
gsearch1.best_estimator_.score(x, y)

0.993680838344135

In [90]:
param_test2 = {'max_depth':range(5,16,2), 'min_samples_split':range(20,101,20)}
gsearch2 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=430, max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test2, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=10)
gsearch2.fit(x,y)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.01684, std: 0.00489, params: {'max_depth': 5, 'min_samples_split': 20},
  mean: -0.01674, std: 0.00480, params: {'max_depth': 5, 'min_samples_split': 40},
  mean: -0.01705, std: 0.00539, params: {'max_depth': 5, 'min_samples_split': 60},
  mean: -0.01639, std: 0.00522, params: {'max_depth': 5, 'min_samples_split': 80},
  mean: -0.01641, std: 0.00506, params: {'max_depth': 5, 'min_samples_split': 100},
  mean: -0.01719, std: 0.00515, params: {'max_depth': 7, 'min_samples_split': 20},
  mean: -0.01751, std: 0.00519, params: {'max_depth': 7, 'min_samples_split': 40},
  mean: -0.01704, std: 0.00510, params: {'max_depth': 7, 'min_samples_split': 60},
  mean: -0.01600, std: 0.00520, params: {'max_depth': 7, 'min_samples_split': 80},
  mean: -0.01662, std: 0.00539, params: {'max_depth': 7, 'min_samples_split': 100},
  mean: -0.01803, std: 0.00518, params: {'max_depth': 9, 'min_samples_split': 20},
  mean: -0.01695, std: 0.00437, params: {'max_depth': 9, 'min_samples_split': 40},
  

In [91]:
gsearch2.best_estimator_.score(x, y)

0.9950559924492098

In [92]:
feature_importance_gbm = list(zip(x.columns, gsearch2.best_estimator_.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance_gbm = np.array(feature_importance_gbm, dtype=dtype)
feature_sort_gbm = np.sort(feature_importance_gbm, order='importance')[::-1]
feature_sort_gbm[:15]

array([(b'GrLivArea', 0.04389743), (b'1stFlrSF', 0.040267  ),
       (b'BsmtUnfSF', 0.03894935), (b'LotArea', 0.0379041 ),
       (b'LotGrBsmtArea', 0.03660855), (b'GarageArea', 0.03331146),
       (b'TotalBsmtSF', 0.0299803 ), (b'YearBuilt', 0.02510116),
       (b'BsmtFinSF1', 0.02426434), (b'GarageYrBlt', 0.02410392),
       (b'LotFrontage', 0.02266381), (b'YearRemodAdd', 0.02176239),
       (b'EnclosedPorch', 0.02032139), (b'OpenPorchSF', 0.02002309),
       (b'MasVnrArea', 0.01872157)],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [97]:
param_test3 = {'min_samples_split':range(80,301,20), 'min_samples_leaf':range(5,66,10)}
gsearch3 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=430,max_depth=7,max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test3, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=10)
gsearch3.fit(x, y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.01695, std: 0.00425, params: {'min_samples_leaf': 5, 'min_samples_split': 80},
  mean: -0.01687, std: 0.00405, params: {'min_samples_leaf': 5, 'min_samples_split': 100},
  mean: -0.01666, std: 0.00444, params: {'min_samples_leaf': 5, 'min_samples_split': 120},
  mean: -0.01677, std: 0.00503, params: {'min_samples_leaf': 5, 'min_samples_split': 140},
  mean: -0.01654, std: 0.00463, params: {'min_samples_leaf': 5, 'min_samples_split': 160},
  mean: -0.01604, std: 0.00443, params: {'min_samples_leaf': 5, 'min_samples_split': 180},
  mean: -0.01625, std: 0.00449, params: {'min_samples_leaf': 5, 'min_samples_split': 200},
  mean: -0.01707, std: 0.00478, params: {'min_samples_leaf': 5, 'min_samples_split': 220},
  mean: -0.01669, std: 0.00428, params: {'min_samples_leaf': 5, 'min_samples_split': 240},
  mean: -0.01695, std: 0.00502, params: {'min_samples_leaf': 5, 'min_samples_split': 260},
  mean: -0.01644, std: 0.00446, params: {'min_samples_leaf': 5, 'min_samples_split': 280},


In [95]:
gsearch3.best_estimator_.score(x, y)

0.9664839328231478

In [96]:
feature_importance_gbm = list(zip(x.columns, gsearch3.best_estimator_.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance_gbm = np.array(feature_importance_gbm, dtype=dtype)
feature_sort_gbm = np.sort(feature_importance_gbm, order='importance')[::-1]
feature_sort_gbm[:15]

array([(b'GrLivArea', 0.05953713), (b'LotGrBsmtArea', 0.04871912),
       (b'1stFlrSF', 0.04864057), (b'GarageArea', 0.0367574 ),
       (b'TotalBsmtSF', 0.03451568), (b'YearBuilt', 0.0303207 ),
       (b'GarageYrBlt', 0.02870639), (b'BsmtFinSF1', 0.02708215),
       (b'OpenPorchSF', 0.02386098), (b'BsmtUnfSF', 0.02073961),
       (b'LotArea', 0.01992681), (b'2ndFlrSF', 0.01902843),
       (b'YearRemodAdd', 0.01724819), (b'LotFrontage', 0.01676905),
       (b'EnclosedPorch', 0.01454544)],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [98]:
param_test4 = {'max_features':range(7,20,2)}
gsearch4 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=430,max_depth=7, min_samples_split=300, min_samples_leaf=15, subsample=0.8, random_state=10),
param_grid = param_test4, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=10)
gsearch4.fit(x, y)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.01670, std: 0.00431, params: {'max_features': 7},
  mean: -0.01620, std: 0.00386, params: {'max_features': 9},
  mean: -0.01608, std: 0.00398, params: {'max_features': 11},
  mean: -0.01623, std: 0.00438, params: {'max_features': 13},
  mean: -0.01582, std: 0.00397, params: {'max_features': 15},
  mean: -0.01584, std: 0.00389, params: {'max_features': 17},
  mean: -0.01605, std: 0.00428, params: {'max_features': 19}],
 {'max_features': 15},
 -0.015818974374064494)

In [99]:
gsearch4.best_estimator_.score(x, y)

0.9714572063105971

In [100]:
feature_importance_gbm = list(zip(x.columns, gsearch4.best_estimator_.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance_gbm = np.array(feature_importance_gbm, dtype=dtype)
feature_sort_gbm = np.sort(feature_importance_gbm, order='importance')[::-1]
feature_sort_gbm[:15]

array([(b'LotGrBsmtArea', 0.0552617 ), (b'GrLivArea', 0.04816826),
       (b'1stFlrSF', 0.04382393), (b'TotalBsmtSF', 0.04070361),
       (b'LotArea', 0.03891458), (b'GarageYrBlt', 0.03502873),
       (b'GarageArea', 0.0347187 ), (b'YearBuilt', 0.03269052),
       (b'BsmtFinSF1', 0.02972023), (b'LotFrontage', 0.02753231),
       (b'BsmtUnfSF', 0.02642578), (b'OpenPorchSF', 0.02453964),
       (b'MasVnrArea', 0.02000579), (b'EnclosedPorch', 0.01936245),
       (b'YearRemodAdd', 0.01870962)],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [101]:
param_test5 = {'subsample':[0.6,0.7,0.75,0.8,0.85,0.9]}
gsearch5 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=430,max_depth=7,min_samples_split=300, min_samples_leaf=15, subsample=0.8, random_state=10,max_features=15),
param_grid = param_test5, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=10)
gsearch5.fit(x, y)
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.01710, std: 0.00421, params: {'subsample': 0.6},
  mean: -0.01632, std: 0.00423, params: {'subsample': 0.7},
  mean: -0.01627, std: 0.00483, params: {'subsample': 0.75},
  mean: -0.01582, std: 0.00397, params: {'subsample': 0.8},
  mean: -0.01591, std: 0.00390, params: {'subsample': 0.85},
  mean: -0.01612, std: 0.00436, params: {'subsample': 0.9}],
 {'subsample': 0.8},
 -0.015818974374064494)

In [112]:
gbm = GradientBoostingRegressor(learning_rate=0.05, n_estimators=860,max_depth=7,min_samples_split=300, min_samples_leaf=15, subsample=0.8, random_state=10,max_features=15)
gbm.fit(x, y)
gbm.score(x, y)

0.9715827477620639

In [113]:
np.sqrt(np.sum((np.array(y)-gbm.predict(x))**2))

2.5720721311110704

In [114]:
feature_importance_gbm = list(zip(x.columns, gbm.feature_importances_))
dtype = [('feature', 'S30'), ('importance', 'float')]
feature_importance_gbm = np.array(feature_importance_gbm, dtype=dtype)
feature_sort_gbm = np.sort(feature_importance_gbm, order='importance')[::-1]
feature_sort_gbm[:15]

array([(b'1stFlrSF', 0.05134955), (b'GrLivArea', 0.04750567),
       (b'LotGrBsmtArea', 0.0474258 ), (b'GarageArea', 0.04162692),
       (b'LotArea', 0.03917006), (b'TotalBsmtSF', 0.03882459),
       (b'GarageYrBlt', 0.03535059), (b'BsmtFinSF1', 0.03190583),
       (b'YearBuilt', 0.03166757), (b'OpenPorchSF', 0.02909522),
       (b'LotFrontage', 0.02613004), (b'BsmtUnfSF', 0.02389583),
       (b'EnclosedPorch', 0.02255974), (b'MasVnrArea', 0.021806  ),
       (b'YearRemodAdd', 0.01831336)],
      dtype=[('feature', 'S30'), ('importance', '<f8')])

In [559]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

lsvc = Lasso(alpha=0.01).fit(x, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(x)
print(lsvc.score(x, y))
x.columns[model.get_support()]

0.8819479424951305


Index(['YearBuilt', 'MasVnrArea', 'TotalBsmtSF', 'GrLivArea', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'Rand_Var', '0', '2', '4', '8'],
      dtype='object')

In [127]:
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score
model_lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005]).fit(x, y)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [133]:
x.head()

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Utilities_NoSeWa,LotGrBsmtArea
0,65.0,8450,2003,2003,196.0,706,0,150,856,856,...,0,0,1,0,0,0,1,0,0,12368772000
1,80.0,9600,1976,1976,0.0,978,0,284,1262,1262,...,0,0,1,0,0,0,1,0,0,15289382400
2,68.0,11250,2001,2002,162.0,486,0,434,920,920,...,0,0,1,0,0,0,1,0,0,18485100000
3,60.0,9550,1915,1970,0.0,216,0,540,756,961,...,0,0,1,0,0,0,0,0,0,12396396600
4,84.0,14260,2000,2000,350.0,655,0,490,1145,1145,...,0,0,1,0,0,0,1,0,0,35888284600


In [131]:
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, x, y, scoring="neg_mean_squared_error", cv = 10))
    return(rmse)

In [132]:
rmse_cv(model_lasso).mean()

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of

0.12809888994660168

In [26]:
from sklearn.svm import SVR

In [27]:
clf = SVR(C=1.0, epsilon=0.2)
clf.fit(x, y)
clf.score(x, y)

0.8106612936700603

In [116]:
y_gbm = gbm.predict(x)
y_xgboost = xgboost.predict(x)
y_rf = randomForest.predict(x)

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Utilities_NoSeWa,LotGrBsmtArea
0,65.0,8450,2003,2003,196.0,706,0,150,856,856,...,0,0,1,0,0,0,1,0,0,12368772000
1,80.0,9600,1976,1976,0.0,978,0,284,1262,1262,...,0,0,1,0,0,0,1,0,0,15289382400
2,68.0,11250,2001,2002,162.0,486,0,434,920,920,...,0,0,1,0,0,0,1,0,0,18485100000
3,60.0,9550,1915,1970,0.0,216,0,540,756,961,...,0,0,1,0,0,0,0,0,0,12396396600
4,84.0,14260,2000,2000,350.0,655,0,490,1145,1145,...,0,0,1,0,0,0,1,0,0,35888284600


In [117]:
from sklearn.neural_network import MLPRegressor


In [118]:
clf = MLPRegressor(hidden_layer_sizes=(1,), max_iter=1000, solver='lbfgs', validation_fraction=0.2)

In [119]:
ensemble_ = pd.DataFrame([y_gbm, y_xgboost, y_rf]).T

In [120]:
ensemble_.shape

(1460, 3)

In [121]:
best_clf = 0
best_score = 0
for i in range(1000):
    clf.fit(ensemble_, y)
    score = clf.score(ensemble_, y)
    if score > best_score:
        best_score = score
        best_clf = clf

In [735]:
[coef.shape for coef in best_clf.coefs_]

[(3, 1), (1, 1)]

In [736]:
best_clf.coefs_

[array([[ 0.90837805],
        [-0.45611193],
        [-1.10160745]]), array([[0.72028207]])]

In [737]:
from sklearn.linear_model import LinearRegression

In [738]:
lm = LinearRegression(fit_intercept=False)

In [739]:
lm.fit(ensemble_, y)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

In [740]:
lm.score(ensemble_, y)

0.991074161238352

In [741]:
lm.coef_

array([-0.72165436,  1.02183541,  0.69995415])

In [778]:
test = pd.read_csv('../x_test.csv')

In [779]:
test.head()

,LotFrontage,LotArea,YearBuilt,MasVnrArea,TotalBsmtSF,GrLivArea,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,...,CentralAirBinary,Has2ndFlr,GarageCarsBinary,FireplacesBinary,TotRmsAbvGrdBinary1,TotRmsAbvGrdBinary2,KitchenAbvGrBinary,BedroomAbvGrBinary,NewHouse,OldHouse
0,28.286562,9.360655,1961,0.0,882.0,896,1961.0,230.312454,140,0,...,False,False,False,False,False,False,True,False,False,True
1,28.542266,9.565704,1958,108.0,1329.0,1329,1958.0,117.231873,393,36,...,False,False,False,False,True,False,True,False,False,True
2,26.730835,9.534595,1997,0.0,928.0,1629,1997.0,165.664191,212,34,...,False,True,True,True,True,False,True,False,True,False
3,27.772150,9.208138,1998,20.0,926.0,1604,1998.0,162.379466,360,36,...,False,True,True,True,True,False,True,False,True,False
4,17.934396,8.518193,1992,0.0,1280.0,1280,1992.0,172.182838,0,82,...,False,False,True,False,False,False,True,False,True,False


In [780]:
x = x.drop(columns=['Rand_Var'])

In [781]:
gbm.fit(x, y)
xgboost.fit(x, y)
randomForest.fit(x, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [782]:
y_gbm_ = gbm.predict(test)
y_xgboost_ = xgboost.predict(test)
y_rf_ = randomForest.predict(test)

In [783]:
z = lm.coef_[0]*y_gbm_ + lm.coef_[1]*y_xgboost_ + lm.coef_[2]*y_rf_

In [775]:
z = 0.5*y_gbm_ + 0.4*y_xgboost_ + 0.1*y_rf_

In [784]:
test_original = pd.read_csv('../test.csv')

In [785]:
final_df = pd.DataFrame({'Id':test_original['Id'], 'SalePrice':np.exp(z)})

In [786]:
final_df.to_csv("../submit.csv", float_format='%f', index=False) #np.savetxt("../submit.csv", np.exp(z), delimiter="\n", fmt='%f')

In [755]:
y.head()

0    12.247694
1    12.109011
2    12.317167
3    11.849398
4    12.429216
Name: SalePrice, dtype: float64

In [759]:
test.head()

,LotFrontage,LotArea,YearBuilt,MasVnrArea,TotalBsmtSF,GrLivArea,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,...,Has2ndFlr,GarageCarsBinary,FireplacesBinary,TotRmsAbvGrdBinary1,TotRmsAbvGrdBinary2,KitchenAbvGrBinary,BedroomAbvGrBinary,NewHouse,OldHouse,Rand_Var
0,28.286562,9.360655,1961,0.0,882.0,896,1961.0,230.312454,140,0,...,False,False,False,False,False,True,False,False,True,69.595973
1,28.542266,9.565704,1958,108.0,1329.0,1329,1958.0,117.231873,393,36,...,False,False,False,True,False,True,False,False,True,40.933021
2,26.730835,9.534595,1997,0.0,928.0,1629,1997.0,165.664191,212,34,...,True,True,True,True,False,True,False,True,False,15.394041
3,27.772150,9.208138,1998,20.0,926.0,1604,1998.0,162.379466,360,36,...,True,True,True,True,False,True,False,True,False,27.638262
4,17.934396,8.518193,1992,0.0,1280.0,1280,1992.0,172.182838,0,82,...,False,True,False,False,False,True,False,True,False,35.952043
